In [1]:
import torch
import torch.nn as nn
import sklearn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.datasets import load_breast_cancer
import onnx
import json
from onnx2pytorch import ConvertModel

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 KB 829.3 kB/s eta 0:00:00 0:00:01
Defaulting to user installation because normal site-packages is not writeable


/home/saimadhavang/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import onnx
from onnx2pytorch import ConvertModel

TRAIN_BS = 32
TEST_BS = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

DLD_DATA = True
train_set = datasets.MNIST('./data', train=True, download=DLD_DATA,
                          transform=transform)
test_set = datasets.MNIST('./data', train=False, download=DLD_DATA,
                          transform=transform)
X_train_tensor = train_set.data
y_train_tensor = train_set.targets
X_test_tensor = test_set.data
y_test_tensor = test_set.targets
train_loader = DataLoader(train_set, batch_size=1, shuffle=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=True)
num_labels = len(np.unique(test_set.targets))
num_inputs = len(X_train_tensor[0])

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:01<00:00, 9015768.43it/s] 


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 675275.77it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 7357918.34it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 5513901.24it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
model = ConvertModel(onnx.load('mnist-net_256x4.onnx'))

/home/saimadhavang/.local/lib/python3.10/site-packages/onnx2pytorch/convert/layer.py:30: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  layer.weight.data = torch.from_numpy(numpy_helper.to_array(weight))


In [5]:
# images, labels = next(iter(train_loader))
# images
neurons=0
relu_layers=0
# Assuming the ReLU layers are named similarly to your PyTorch model, you can add hooks like this
for name, layer in model.named_modules():
  if isinstance(layer, nn.Linear):
    neurons=layer.in_features
  if isinstance(layer, torch.nn.ReLU):
    relu_layers+=1

states={}
for i in range(num_labels):
  states[i]=[]

with torch.no_grad():
  for i, t in train_loader:
      t=t.item()
      i=i.float().to(device)
      out=get_relu_activations(model,i)
      states[t].append(out.detach().cpu().numpy().tolist())
print(len(states))
print(len(states[0]))

NameError: name 'get_relu_activations' is not defined

In [ ]:
for i in states:
  states[i]=np.array(states[i])
label_naps = get_label_naps(states)

In [ ]:
i=0
neurons=0
# Assuming the ReLU layers are named similarly to your PyTorch model, you can add hooks like this
for name, layer in model.named_modules():
  if isinstance(layer, nn.Linear):
    neurons=layer.in_features
  if isinstance(layer, torch.nn.ReLU):
      i=i+1
relu_layers=i

/usr/local/lib/python3.10/dist-packages/onnx2pytorch/convert/layer.py:30: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  layer.weight.data = torch.from_numpy(numpy_helper.to_array(weight))
